In [1]:
import base64
from lxml import etree
from xml.etree import ElementTree
from gvm.connections import TLSConnection, SSHConnection 
from gvm.protocols.latest import Gmp
from gvm.transforms import EtreeTransform
from gvm.xml import pretty_print
from pygvm import Pygvm
import time


In [2]:
## connection = SSHConnection(hostname='223.193.36.146', port=2222, timeout=25, username='test', password='test')
connection = TLSConnection(hostname='192.168.100.148', port=9390, cafile='../ca/cacert.pem', 
                                                                certfile='../ca/clientcert.pem', 
                                                                keyfile='../ca/clientkey.pem')
transform = EtreeTransform()
gmp = Gmp(connection, transform=transform)
pyg = Pygvm(gmp=gmp, username='admin', passwd='cnic.cn')
print(pyg.checkauth())
print(pyg.get_version())

True
22.4


In [3]:
res = pyg.list_scanners()
print(res)
print(res[0]['name'])
print(res[0]['@id'])
print(res[1]['name'])
print(res[1]['@id'])

[{'owner': {'name': None}, 'name': 'CVE', 'comment': None, 'creation_time': '2023-10-17T11:19:29Z', 'modification_time': '2023-10-17T11:19:29Z', 'writable': '1', 'in_use': '0', 'permissions': {'permission': [{'name': 'get_scanners'}, {'name': 'get_scanners'}, {'name': 'get_scanners'}, {'name': 'get_scanners'}]}, 'host': None, 'port': '0', 'type': '3', 'ca_pub': None, 'credential': {'name': None, 'type': None, 'trash': '0', '@id': ''}, '@id': '6acd0832-df90-11e4-b9d5-28d24461215b'}, {'owner': {'name': None}, 'name': 'OpenVAS Default', 'comment': None, 'creation_time': '2023-10-17T11:19:29Z', 'modification_time': '2023-10-17T18:37:15Z', 'writable': '1', 'in_use': '1', 'permissions': {'permission': [{'name': 'get_scanners'}, {'name': 'get_scanners'}, {'name': 'get_scanners'}, {'name': 'get_scanners'}]}, 'host': '/run/ospd/ospd.sock', 'port': '0', 'type': '2', 'ca_pub': None, 'credential': {'name': None, 'type': None, 'trash': '0', '@id': ''}, '@id': '08b69003-5fc2-4037-a479-93b440211c73'}

In [ ]:
res = pyg.delete_scanner(scanner_id="a138b49a-5e0a-44d9-916b-8bceb8e8a282")
print(res)

### 创建目标

In [4]:
#33d0cd82-57c6-11e1-8ed1-406186ea4fc5
res = pyg.create_target('test_57', hosts=['223.193.36.146'], port_list_id='33d0cd82-57c6-11e1-8ed1-406186ea4fc5')
print(res['@id'])

d2416779-0232-4bac-9f8b-f61637e7a719


### 创建任务

In [7]:
tasks = []
for i in range(1):
# 创建任务
# '@id' : 'c11a8fbf-4403-4622-ac58-0ec85919bde9'
    time.sleep(1)
    res = pyg.create_task(name=f"test_for_146_for_{i}",target_id='d2416779-0232-4bac-9f8b-f61637e7a719', 
                          config_id='daba56c8-73ec-11df-a475-002264764cea', 
                          scanner_id='08b69003-5fc2-4037-a479-93b440211c73',
                            preferences={'assets_min_qod' : 30})
    tasks.append(res['@id'])
    print(f"task {res['@id']} append.")
    try:
        res = pyg.start_task(task_id=res['@id'])
        print(res)
    except:
        print(f"task {res['@id']} start failed.")

task 1f320065-ce63-408d-a526-da87b5fa5c4d append.
{'report_id': '77116dd5-9e9f-48c7-9faa-026f53f9cae7', '@status': '202', '@status_text': 'OK, request submitted'}


In [11]:
res = pyg.list_tasks()
for re in res:
    print(re['@id'])

In [5]:
tasks = pyg.list_tasks()
for task in tasks:
    try:
        pyg.delete_task(task_id=task['@id'])
        print(f"task {task['@id']} deleted.")
    except:
        print(f"task {res['@id']} deleted failed.")

task 1ea26908-7cc2-4fc1-8a96-0e43a882c33c deleted.
task 5e5d351c-4a25-41d5-b872-c6c6be8a8dd7 deleted.
task 1f320065-ce63-408d-a526-da87b5fa5c4d deleted.


### 获取任务进度
#### task的状态status包括Requested / Queued / Running / Done / Failed / Interapted
#### 可以通过 progress得到task进度

In [6]:
# tasks_d = pyg.list_tasks()
# tasks = []
# for task_d in tasks_d:
#     tasks.append(task_d['@id'])
print(time.asctime())
res = pyg.get_task(task_id=tasks[0])
print(res)
while True:
    for task in tasks[:]:
        res = pyg.get_task(task_id=task)
        print(f"task {task} progress is {res['progress']}")
        if(res['progress'] == '-1'):
            tasks.remove(task)
            print(f"task {task} finished.")
    if(len(tasks) == 0): break
    time.sleep(60)
print(time.asctime())

Fri Jan 19 09:19:13 2024
{'owner': {'name': 'admin'}, 'name': 'test_for_146_for_0', 'comment': None, 'creation_time': '2024-01-19T01:18:42Z', 'modification_time': '2024-01-19T01:18:42Z', 'writable': '1', 'in_use': '1', 'permissions': {'permission': {'name': 'Everything'}}, 'alterable': '0', 'usage_type': 'scan', 'config': {'name': 'Full and fast', 'trash': '0', '@id': 'daba56c8-73ec-11df-a475-002264764cea'}, 'target': {'name': 'test_57', 'trash': '0', '@id': 'd2416779-0232-4bac-9f8b-f61637e7a719'}, 'hosts_ordering': None, 'scanner': {'name': 'OpenVAS Default', 'type': '2', 'trash': '0', '@id': '08b69003-5fc2-4037-a479-93b440211c73'}, 'status': 'Running', 'progress': '0', 'report_count': {'finished': '0', '#text': '1'}, 'trend': None, 'schedule': {'name': None, 'trash': '0', '@id': ''}, 'schedule_periods': '0', 'current_report': {'report': {'timestamp': '2024-01-19T01:18:42Z', 'scan_start': '2024-01-19T01:19:01Z', 'scan_end': None, '@id': '60ee773e-5c34-48df-9142-27750d9bab76'}}, 'obser

Wed Jan  3 12:18:13 2024


### 获取report

In [15]:
from utils import lxml_to_dict


res = pyg.list_reports(task_id='5e5d351c-4a25-41d5-b872-c6c6be8a8dd7')
print(res[0]['@id'])
reportid = res[0]['@id']
# 获取report
# filter_str="apply_overrides=0 levels=hml rows=100 min_qod=30 first=1 sort-reverse=severity host~"223.193.36.146 result_hosts_only=1"
res = pyg.get_report(report_id=reportid, report_format_name='PDF', 
                     filter_str='apply_overrides=0 levels=hml rows=1000 min_qod=30 first=1 sort-reverse=severity')
# res = lxml_to_dict(res)
# print(res['report'])
print(res)
# pretty_print(res)
# res = base64.b64decode(res)
# print(res)
# with open('./report1.pdf', 'wb+') as f:
#     f.write(res)

60ee773e-5c34-48df-9142-27750d9bab76
...


In [11]:
res = pyg.list_report_formats()
for re in res:
    print(re)

{'owner': {'name': 'admin'}, 'name': 'Anonymous XML', 'comment': None, 'creation_time': '2023-11-16T04:41:25Z', 'modification_time': '2023-12-04T02:39:22Z', 'writable': '1', 'in_use': '0', 'permissions': {'permission': {'name': 'Everything'}}, 'extension': 'xml', 'content_type': 'text/xml', 'summary': 'Anonymous version of the raw XML report. Version 20200827.', 'description': 'Anonymized scan report in GMP XML format. Version 20200827.', 'predefined': '1', 'trust': {'time': '2023-11-16T04:41:25Z', '#text': 'yes'}, 'active': '1', '@id': '5057e5cc-b825-11e4-9d0e-28d24461215b'}
{'owner': {'name': 'admin'}, 'name': 'CSV Results', 'comment': None, 'creation_time': '2023-11-16T04:41:25Z', 'modification_time': '2023-12-04T02:39:22Z', 'writable': '1', 'in_use': '0', 'permissions': {'permission': {'name': 'Everything'}}, 'extension': 'csv', 'content_type': 'text/csv', 'summary': 'CSV result list. Version 20231121.', 'description': 'List of results as comma separated values. Version 20231121. T

In [14]:
print(len('4732ef12-42fd-4c4c-82d6-f044d6018021'))

36
